In [42]:
import ast
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import torch
import gc
import numpy as np
from tqdm import tqdm

tqdm.pandas()

# RAG: Query based retrival using embeddings
### Overview
The goal of this step is to filter and rank articles based on their embedding similarity scores to a user-provided prompt. This is done using a pre-trained model to convert the prompt into embeddings, and then comparing these embeddings with the embeddings of articles stored in a DataFrame. <br>

The open source model is the same used before in the preprocessing step, which is [`jinaai/jina-embeddings-v3`](https://huggingface.co/jinaai/jina-embeddings-v3).

#### Possibile future improvements
The current filtering approach based on user queries can be expanded by incorporating additional criteria such as number of downloads, likes, or publication date. This a simple implementation which does not require any computational resources.<br>
Moreover, the retrieval performance can be improved by experimenting with alternative similarity metrics such as cosine similarity or Euclidean distance or trying with another model.


In [43]:
embedding_model = "jinaai/jina-embeddings-v3"


def convert_prompt_to_embedding(prompt):
    """
    Converts text prompt to embeddings using a pre-trained model.

    Args:
        prompt (str): Input text to convert to embeddings

    Returns:
        numpy.ndarray: Embedding vector representation of the input text
    """
    model = AutoModel.from_pretrained(embedding_model, trust_remote_code=True).to("cpu")  # for now cpu
    embedding = model.encode(prompt, task="text-matching")
    torch.cuda.empty_cache()
    gc.collect()
    return embedding


def compute_score(embeddings, prompt):
    models_embedding = np.array(embeddings)
    return np.dot(models_embedding, prompt)


def filter_by_score(data, prompt, range=10):
    """
    Filters and ranks models based on embedding similarity scores.

    Args:
        data (pd.DataFrame): DataFrame containing 'embeddings' column
        prompt (numpy.ndarray): Prompt embedding to compare against
        range (int, optional): Number of top models to return. Defaults to 10.

    Returns:
        pd.DataFrame: Top N articles sorted by similarity score
    """

    data['embeddings'] = data['embeddings'].apply(
        lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else x)  #da sistemare

    data['score'] = data['embeddings'].progress_apply(lambda x: compute_score(x, prompt))
    data.sort_values(by='score', ascending=False, inplace=True)
    return data.head(range)


def filter_by_user_prompt(data, user_prompt):
    prompt_embedding = convert_prompt_to_embedding(user_prompt)
    data = filter_by_score(data, prompt_embedding)
    return data

In [41]:
df = pd.read_csv("huggingface_models_embeddings.csv")
df.head()

,model_id,base_model,author,readme_file,license,language,downloads,likes,tags,pipeline_tag,library_name,created_at,embeddings
0,deepseek-ai/DeepSeek-R1-0528,NaN,deepseek-ai,DeepSeek-R1-0528\nPaper Link👁️\nIntroduction\n...,mit,NaN,47886,1702,"transformers, safetensors, deepseek_v3, text-g...",text-generation,transformers,2025-05-28 09:46:42+00:00,"[0.1295202076435089, -0.03313147649168968, 0.0..."
1,deepseek-ai/DeepSeek-R1-0528-Qwen3-8B,NaN,deepseek-ai,DeepSeek-R1-0528\nPaper Link👁️\nIntroduction\n...,mit,NaN,72595,629,"transformers, safetensors, qwen3, text-generat...",text-generation,transformers,2025-05-29 11:07:47+00:00,"[0.11936206370592117, -0.02954871952533722, 0...."
2,google/gemma-3n-E4B-it-litert-preview,NaN,google,[!Note]\nThis repository corresponds to the Pr...,gemma,NaN,0,882,"image-text-to-text, arxiv:1905.07830, arxiv:19...",image-text-to-text,NaN,2025-05-18 19:24:14+00:00,"[0.12827537953853607, -0.11014561355113983, 0...."
3,osmosis-ai/Osmosis-Structure-0.6B,NaN,osmosis-ai,Osmosis-Structure-0.6B: Small Language Model f...,apache-2.0,NaN,554,229,"safetensors, gguf, license:apache-2.0, endpoin...",NaN,NaN,2025-05-28 15:39:48+00:00,"[0.1758263260126114, 0.06860576570034027, 0.04..."
4,ResembleAI/chatterbox,NaN,ResembleAI,Chatterbox TTS\nMade with ❤️ by\nWe're excit...,mit,['en'],0,560,"chatterbox, text-to-speech, speech generation,...",text-to-speech,chatterbox,2025-04-24 12:03:33+00:00,"[0.24660152196884155, 0.07107454538345337, 0.0..."


### Example with models from Hugging Face

In [31]:
# Model prompts
user_prompts = [
    "What is the best model for text generation?",
    "Which model should I use for sentiment analysis?",
    "Find top models for image classification.",
    "Best models for summarization tasks?",
    "What are the newest models for code generation?",
    "Top-performing models for question answering?",
    "Which models support Italian language?",
    "Best lightweight models for mobile deployment.",
    "Which models are most popular on Hugging Face?",
    "Find models optimized for speed and low latency."
]

# Log results
with open("models_results_log.txt", "w", encoding="utf-8") as log_file:
    for i, prompt in enumerate(user_prompts, 1):
        log_file.write(f"\n--- Query {i}: {prompt} ---\n")
        filtered_df = filter_by_user_prompt(df, prompt)
        for _, row in filtered_df.iterrows():
            log_file.write(f"\n{row['model_id']} | score: {row['score']:.4f}\n")
            log_file.write(f"Author: {row['author']}\n")
            log_file.write(f"Pipeline Tag: {row['pipeline_tag']}\n")
            log_file.write(f"ReadmeFile, first 500 characters: {row['readme_file'][:500]}\n")
            log_file.write("-" * 50 + "\n")
        log_file.write("\n")

KeyboardInterrupt: 

### Example with datasets from Hugging Face

In [44]:
df = pd.read_csv("Output/datasets_hg_embeddings.csv")
print(df.shape)

(30000, 13)


In [37]:
df.head()

,dataset_id,author,created_at,readme_file,downloads,likes,tags,language,license,multilinguality,size_categories,task-categories,embeddings
0,Nyameri/AIXDR,Nyameri,2023-11-03 05:19:15+00:00,# Dataset Card for Dataset Name\n\nThis datase...,28,1,"['task_categories:summarization', 'task_catego...",NaN,mit,NaN,['1K<n<10K'],"['summarization', 'feature-extraction']",[ 0.09661677 -0.13288344 0.0624015 ... -0.01...
1,agicorp/MetaMathQA,agicorp,2024-03-23 08:26:51+00:00,View the project page:\nhttps://meta-math.gith...,13,1,"['license:mit', 'size_categories:100K<n<1M', '...",NaN,mit,NaN,NaN,NaN,[ 0.24795215 -0.03745317 0.13433613 ... -0.02...
2,avsolatorio/medi-data,avsolatorio,2024-01-26 03:49:25+00:00,# MEDI dataset\n\nThis dataset was used in the...,15,0,"['size_categories:1M<n<10M', 'format:parquet',...",NaN,NaN,NaN,NaN,NaN,[ 0.11949765 -0.10292578 0.08450241 ... -0.02...
3,Yuanze/Olympus,Yuanze,2025-04-02 10:05:03+00:00,# Olympus: A Universal Task Router for Compute...,179,5,"['task_categories:question-answering', 'task_c...",['en'],apache-2.0,NaN,['100K<n<1M'],"['question-answering', 'visual-question-answer...",[ 0.04112962 -0.05431836 0.14046644 ... -0.02...
4,Laxhar/noob-wiki,Laxhar,2024-11-14 07:48:42+00:00,# Noob SDXL Wiki\n\nThis is the WIKI database ...,2596,85,"['task_categories:text-to-image', 'language:en...",['en'],apache-2.0,NaN,NaN,['text-to-image'],[ 0.06410673 0.02687506 0.06094188 ... -0.00...


In [45]:
# Dataset prompts
user_prompts = [
    "Which datasets are best for sentiment analysis in Italian?",
    "Find large-scale datasets with multilingual support and open licenses.",
    "What are the top trending datasets for text summarization?",
    "Datasets with detailed README files and active contributors.",
    "Show datasets suitable for low-resource language modeling.",
    "Which datasets support image classification tasks and are under 1GB?",
    "List recently created datasets for question answering in biomedical domain.",
    "Find datasets curated for cross-lingual classification tasks with labeled examples and language identifiers.",
    "Find high-quality datasets for code generation with permissive licenses.",
    "List benchmark NLP datasets commonly used in academic research and model evaluation."
]

# Log results
with open("Results/datasets_results.txt", "w", encoding="utf-8") as log_file:
    for i, prompt in enumerate(user_prompts, 1):
        log_file.write(f"\n--- Query {i}: {prompt} ---\n")
        filtered_df = filter_by_user_prompt(df, prompt)
        for _, row in filtered_df.iterrows():
            log_file.write(f"\n{row['dataset_id']} | score: {row['score']:.4f}\n")
            log_file.write(f"Author: {row['author']}\n")
            log_file.write(f"Task Categories: {row['task-categories']}\n")
            log_file.write(f"ReadmeFile, first 500 characters: {row['readme_file'][:500]}\n")
            log_file.write("-" * 50 + "\n")
        log_file.write("\n")

100%|██████████| 30000/30000 [00:00<00:00, 219499.77it/s]
